In [2]:
pip install tokopaedi

Note: you may need to restart the kernel to use updated packages.


In [5]:
import time
import random
import json
from tokopaedi import search, get_reviews
import pandas as pd

# --- Konfigurasi ---
KEYWORDS = [
    "elektronik", "handphone", "komputer", "fashion pria", "fashion wanita",
    "ibu dan bayi", "kecantikan", "kesehatan", "rumah tangga", "olahraga",
    "hobi", "makanan", "minuman", "sekolah",
]

MAX_REVIEWS_PER_PRODUCT = 19   # review per produk
MAX_PRODUCTS_PER_KEYWORD = 10  # produk per keyword
SLEEP_SEARCH = (3.0, 5.0)
SLEEP_DETAIL = (1.2, 2.0)
RETRY_PER_ITEM = 2

all_reviews = []  # akan berisi string teks ulasan saja

def sleep_range(rng):
    time.sleep(random.uniform(*rng))

def get_attr(obj, *names):
    """Ambil atribut dari object (beragam implementasi library)."""
    for n in names:
        if hasattr(obj, n):
            return getattr(obj, n)
    if hasattr(obj, "json"):
        d = obj.json()
        for n in names:
            if n in d:
                return d[n]
    # kalau obj itu dict-like
    try:
        if isinstance(obj, dict):
            for n in names:
                if n in obj:
                    return obj[n]
    except Exception:
        pass
    return None

def extract_review_text(review_obj):
    """Coba ekstrak teks review dari ProductReview object / dict."""
    # 1) jika ada method .json()
    data = None
    if hasattr(review_obj, "json"):
        try:
            data = review_obj.json()
        except Exception:
            data = None
    # 2) kalau dict
    if data is None and isinstance(review_obj, dict):
        data = review_obj
    # 3) fallback: vars()
    if data is None:
        try:
            data = vars(review_obj)
        except Exception:
            data = None

    # search kemungkinan key untuk teks ulasan
    if isinstance(data, dict):
        for key in ("content", "review", "message", "text", "review_text", "review_content", "comment"):
            if key in data and data[key]:
                return str(data[key]).strip()

    # kalau belum ketemu, cek attribute langsung (mis. r.content)
    for attr in ("content", "review", "message", "text", "review_text", "review_content", "comment"):
        if hasattr(review_obj, attr):
            val = getattr(review_obj, attr)
            if val:
                return str(val).strip()

    # terakhir coba str(review_obj)
    try:
        s = str(review_obj).strip()
        return s if len(s) > 0 else None
    except Exception:
        return None

# --- Main loop ---
for keyword in KEYWORDS:
    print(f"\n🔍 Mencari produk untuk keyword: {keyword}")
    try:
        # pakai parameter yang benar: max_result (bukan limit)
        results = search(keyword, max_result=MAX_PRODUCTS_PER_KEYWORD, debug=False)
        products = list(results)
    except TypeError:
        # fallback: kalau signature berbeda
        results = search(keyword, debug=False)
        products = list(results)[:MAX_PRODUCTS_PER_KEYWORD]
    except Exception as e:
        print(f"❌ Gagal search '{keyword}': {e}")
        continue

    print(f"  ➜ Ditemukan {len(products)} produk (dibatasi {MAX_PRODUCTS_PER_KEYWORD})")
    sleep_range(SLEEP_SEARCH)

    for p in products:
        # ambil url / product_id jika ada (tidak wajib untuk hasil akhir)
        purl = get_attr(p, "url", "product_url")
        pid  = get_attr(p, "product_id", "id")

        if not purl and not pid:
            # kalau tidak ada keduanya, skip produk ini
            continue

        # try ambil review (retry ringan)
        for attempt in range(RETRY_PER_ITEM):
            try:
                # beberapa versi get_reviews terima max_count, beberapa tidak
                try:
                    if purl:
                        reviews = get_reviews(url=purl, max_count=MAX_REVIEWS_PER_PRODUCT, debug=False)
                    else:
                        reviews = get_reviews(product_id=pid, max_count=MAX_REVIEWS_PER_PRODUCT, debug=False)
                except TypeError:
                    # fallback tanpa max_count
                    if purl:
                        reviews = get_reviews(url=purl, debug=False)
                    else:
                        reviews = get_reviews(product_id=pid, debug=False)

                if not reviews:
                    # tidak ada review untuk produk ini
                    break

                # ekstrak teks tiap review (dan simpan hanya teksnya)
                for r in reviews:
                    txt = extract_review_text(r)
                    if txt:
                        all_reviews.append(txt)

                # selesai untuk produk ini
                break

            except Exception as e:
                print(f"   ⚠️ Gagal fetch review (attempt {attempt+1}) untuk produk (pid/url) — {e}")
                time.sleep(random.uniform(1.5, 3.5))

        sleep_range(SLEEP_DETAIL)

# Simpan hasil: hanya teks review
print(f"\n✅ Selesai. Total teks review terkumpul: {len(all_reviews)}")

# simpan ke CSV 1 kolom
df = pd.DataFrame(all_reviews, columns=["review"])
df.to_csv("reviews_only.csv", index=False, encoding="utf-8-sig")

# simpan ke JSON (list string)
with open("reviews_only.json", "w", encoding="utf-8") as f:
    json.dump(all_reviews, f, ensure_ascii=False, indent=2)

# print contoh 10 review pertama
print("\nContoh 10 review pertama:")
for i, t in enumerate(all_reviews[:10], 1):
    print(f"{i}. {t[:200]}")


🔍 Mencari produk untuk keyword: elektronik
  ➜ Ditemukan 16 produk (dibatasi 10)

🔍 Mencari produk untuk keyword: handphone
  ➜ Ditemukan 16 produk (dibatasi 10)

🔍 Mencari produk untuk keyword: komputer
  ➜ Ditemukan 16 produk (dibatasi 10)

🔍 Mencari produk untuk keyword: fashion pria
  ➜ Ditemukan 16 produk (dibatasi 10)

🔍 Mencari produk untuk keyword: fashion wanita
  ➜ Ditemukan 16 produk (dibatasi 10)

🔍 Mencari produk untuk keyword: ibu dan bayi
  ➜ Ditemukan 16 produk (dibatasi 10)

🔍 Mencari produk untuk keyword: kecantikan
  ➜ Ditemukan 16 produk (dibatasi 10)

🔍 Mencari produk untuk keyword: kesehatan
  ➜ Ditemukan 16 produk (dibatasi 10)

🔍 Mencari produk untuk keyword: rumah tangga
  ➜ Ditemukan 16 produk (dibatasi 10)

🔍 Mencari produk untuk keyword: olahraga
  ➜ Ditemukan 16 produk (dibatasi 10)

🔍 Mencari produk untuk keyword: hobi
  ➜ Ditemukan 16 produk (dibatasi 10)

🔍 Mencari produk untuk keyword: makanan
Traceback (most recent call last):
  File "C:\Users\vivin\a